# Sequencing data analysis

### IMPORTANT: Please make sure that your are using the bash kernel to run this notebook.
#### (Do this at the beginning of every session) ###

### This notebook covers analysis of DNA sequencing data from raw files to processed signals.

Although this analysis is for ATAC-seq data, many of the steps (especially the first section) are the same for other types of DNA sequencing experiments.

We'll be doing the analysis in Bash, which is the standard language for UNIX command-line scripting.

The steps in the analysis pipeline that are covered in this notebook are indicated below:
![Sequencing Data Analysis 1](images/part1.png)

## Part 1: Setting up the data

We start with raw `.fastq.gz` files, which are provided by the sequencing instrument. For each DNA molecule (read) that was sequenced, they provide the nucleotide sequence, and information about the quality of the signal of that nucleotide.

In [14]:
### Set up variables storing the location of our data
### The proper way to load your variables is with the ~/.bashrc command, but this is very slow in iPython 
export SUNETID="$(whoami)"
export WORK_DIR="/srv/scratch/training_camp/work/${SUNETID}"
export DATA_DIR="${WORK_DIR}/data"
[[ ! -d ${WORK_DIR}/data ]] && mkdir "${WORK_DIR}/data"
export SRC_DIR="${WORK_DIR}/src"
[[ ! -d ${WORK_DIR}/src ]] && mkdir -p "${WORK_DIR}/src"
export METADATA_DIR="/srv/scratch/training_camp/metadata"
export AGGREGATE_DATA_DIR="/srv/scratch/training_camp/data"
export AGGREGATE_ANALYSIS_DIR="/srv/scratch/training_camp/data"
export YEAST_DIR="/srv/scratch/training_camp/saccer3"
export TMP="${WORK_DIR}/tmp"
export TEMP=$TMP
export TMPDIR=$TMP
[[ ! -d ${TMP} ]] && mkdir -p "${TMP}"



Now, let's check exactly which fastqs we have (we copied these from $AGGREGATE_DATA_DIR to your personal $DATA_DIR in the last tutorial):

(recall that the `ls` command lists the contents of a directory)

In [ ]:
ls $DATA_DIR

As a sanity check, we can also look at the size and last edited time of some of the fastqs by addind `-lrth` to the `ls` command:

In [15]:
ls -lrth $DATA_DIR

total 383M
-rwxrwxr-x 1 ubuntu ubuntu  32M Sep  8 02:05 WT-SCD-Rep1_R1_001.fastq.gz
-rwxrwxr-x 1 ubuntu ubuntu  28M Sep  8 02:05 WT-SCD-Rep1_R2_001.fastq.gz
-rwxrwxr-x 1 ubuntu ubuntu 168M Sep  8 02:06 WT-SCD-Rep2_R1_001.fastq.gz
-rwxrwxr-x 1 ubuntu ubuntu 157M Sep  8 02:06 WT-SCD-Rep2_R2_001.fastq.gz


Let's also inspect the format of one of the fastqs. Notice that each read takes up 4 lines:
1. the read name
2. the read's nucleotide sequence
3. a '+' to indicate the record contains another line
4. a quality score for each base (a number encoded as a letter)

In [16]:
zcat $(ls $DATA_DIR/*gz | head -n 1) | head -n 8

@NS500418:691:HTFJ7AFXX:1:11101:13955:1071 1:N:0:CTCTCTAC+GCGATCTA
TCTCTATGATGGTAATAGGCAAACATCGGGCGTACCTTAAAAGTCTTAGACATCACATAAACTGTCTCTTATACAC
+
AAAAAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAEEEEEEEEEEEEAEEEAEEEEEEEEE
@NS500418:691:HTFJ7AFXX:1:11101:20718:1090 1:N:0:CTCTCTAC+GCGATCTA
CCCCCTCCCATTACAAACTAAAATCTTACTTTTATTTTCTTTTGCCCTCTCTGTCGCCTGTCTCTTATACACATCT
+
AAAAAEEEEEEEEEEEEAEEEEEEEEEEAEEEEEEEEE<EEEEEEEEAE/EEEEE/EEEEAE<6EEEAEEEAEEAE

gzip: stdout: Broken pipe


## Part 2:ATAC-seq data processing

The ENCODE consortium (https://www.encodeproject.org/) uses a standard ATAC-seq data processing pipeline, which can be downloaded here: https://github.com/ENCODE-DCC/atac-seq-pipeline

This pipeline is pre-installed on this computer and can be executed by running the **atac.bds** script. 



In [17]:
export PATH=$PATH:/opt/atac_dnase_pipelines
atac.bds --help

== atac pipeline settings
	-type <string>                   : Type of the pipeline. atac-seq or dnase-seq (default: atac-seq).
	-dnase_seq <bool>                : DNase-Seq (no tn5 shifting).
	-align <bool>                    : Align only (no MACS2 peak calling or IDR or ataqc analysis).
	-subsample_xcor <string>         : # reads to subsample for cross corr. analysis (default: 25M).
	-subsample <string>              : # reads to subsample exp. replicates. Subsampled tagalign will be used for steps downstream (default: 0; no subsampling).
	-true_rep <bool>                 : No pseudo-replicates.
	-no_ataqc <bool>                 : No ATAQC
	-no_xcor <bool>                  : No Cross-correlation analysis.
	-csem <bool>                     : Use CSEM for alignment.
	-smooth_win <string>             : Smoothing window size for MACS2 peak calling (default: 150).
	-idr_thresh <real>               : IDR threshold : -log_10(score) (default: 0.1).
	-ENCODE3 <bool>                  : Force to 

Though the pipeline is highly customizable and all the customizations might seem a bit confusing at first, do not worry -- for our purposes, the default settings will suffice. You will run the pipeline on your two experiments. Fill in the names of the FASTQ files corresponding to your two experiments below, as well as the name of the ouptut directory to store the processed data. 

In [22]:
#You can find the experiment names in the file $METADATA_DIR/TC2018_samples.tsv.
#Look under the column labeled "ID"
#example: 

#export experiment1="WT_ethanol_1"
#export experiment2="asf1_ethanol_1"

export experiment1="WT-SCD-Rep1"
export experiment2="WT-SCD-Rep2"

#Create directories to store outputs from the pipeline
#We will store the outputs in the $WORK_DIR
export outdir1=$WORK_DIR/$experiment1\_out 
export outdir2=$WORK_DIR/$experiment2\_out
mkdir $outdir1
mkdir $outdir2


Now, kick off the pipeline! 

In [26]:
#first experiment:
echo "bds_scr $experiment1 $experiment1.log atac.bds -out_dir $outdir1 -species saccer3 -fastq1_1 $DATA_DIR/$experiment1\_R1_001.fastq.gz -fastq1_2 $DATA_DIR/$experiment1\_R2_001.fastq.gz -nth 4"
bds_scr $experiment1 $outdir1/log.txt atac.bds -out_dir $outdir1 -species saccer3 -fastq1_1 $DATA_DIR/$experiment1\_R1_001.fastq.gz -fastq1_2 $DATA_DIR/$experiment1\_R2_001.fastq.gz -nth 4 

/opt/.bds/bds_scr atac.bds -out_dir /srv/scratch/training_camp/work/ubuntu/WT-SCD-Rep1_out -species saccer3 -fastq1_1 /srv/scratch/training_camp/work/ubuntu/data/WT-SCD-Rep1\_R1_001.fastq.gz -fastq1_2 /srv/scratch/training_camp/work/ubuntu/data/WT-SCD-Rep1\_R2_001.fastq.gz -nth 4
[SCR_NAME] : atac.bds.BDS
[HOST] : ip-172-31-26-41.us-west-1.compute.internal
[LOG_FILE_NAME] : /home/ubuntu/training_camp/workflow_notebooks/atac.bds.BDS.log
[BDS_PARAM] :  -out_dir /srv/scratch/training_camp/work/ubuntu/WT-SCD-Rep1_out -species saccer3 -fastq1_1 /srv/scratch/training_camp/work/ubuntu/data/WT-SCD-Rep1_R1_001.fastq.gz -fastq1_2 /srv/scratch/training_camp/work/ubuntu/data/WT-SCD-Rep1_R2_001.fastq.gz -nth 4



In [29]:
#second experiment:
echo "bds_scr $experiment2 $experiment2.log atac.bds -out_dir $outdir2 -species saccer3 -fastq1_1 $DATA_DIR/$experiment2_R1_001.fastq.gz -fastq1_2 $DATA_DIR/$experiment2_R2_001.fastq.gz -nth 4"
bds_scr $experiment2 $outdir2/log.txt atac.bds -out_dir $outdir2 -species saccer3 -fastq1_1 $DATA_DIR/$experiment2\_R1_001.fastq.gz -fastq1_2 $DATA_DIR/$experiment2\_R2_001.fastq.gz -nth 4 

bds_scr WT-SCD-Rep1 WT-SCD-Rep1.log atac.bds -out_dir /srv/scratch/training_camp/work/ubuntu/WT-SCD-Rep2_out -species saccer3 -fastq1_1 /srv/scratch/training_camp/work/ubuntu/data/.fastq.gz -fastq1_2 /srv/scratch/training_camp/work/ubuntu/data/.fastq.gz -nth 4
[SCR_NAME] : WT-SCD-Rep1.BDS
[HOST] : ip-172-31-26-41.us-west-1.compute.internal
[LOG_FILE_NAME] : WT-SCD-Rep1.log
[BDS_PARAM] :  atac.bds -out_dir /srv/scratch/training_camp/work/ubuntu/WT-SCD-Rep2_out -species saccer3 -fastq1_1 /srv/scratch/training_camp/work/ubuntu/data/WT-SCD-Rep2_R1_001.fastq.gz -fastq1_2 /srv/scratch/training_camp/work/ubuntu/data/WT-SCD-Rep2_R2_001.fastq.gz -nth 4



The pipeline may run for an hour or so, so meanwhile, we will learn more about what it's doing under the hood. 
If you want to check on the progress, you can examine the log file generated by the pipeline:


In [28]:
tail $experiment1.log

tail: cannot open ‘/srv/scratch/training_camp/work/ubuntu/WT-SCD-Rep1_out/log.txt’ for reading: No such file or directory


In [ ]:
tail $experiment2.log

## Part 3: Examining the pipeline output

The pipeline consists of multiple modules, with output files that include the following: 

```
out                               # root dir. of outputs
│
├ *report.html                    #  HTML report
├ *tracks.json                    #  Tracks datahub (JSON) for WashU browser
├ ENCODE_summary.json             #  Metadata of all datafiles and QC results
│
├ align                           #  mapped alignments
│ ├ rep1                          #   for true replicate 1 
│ │ ├ *.trim.fastq.gz             #    adapter-trimmed fastq
│ │ ├ *.bam                       #    raw bam
│ │ ├ *.nodup.bam (E)             #    filtered and deduped bam
│ │ ├ *.tagAlign.gz               #    tagAlign (bed6) generated from filtered bam
│ │ ├ *.tn5.tagAlign.gz           #    TN5 shifted tagAlign for ATAC pipeline (not for DNase pipeline)
│ │ └ *.*M.tagAlign.gz            #    subsampled tagAlign for cross-corr. analysis
│ ├ rep2                          #   for true repilicate 2
│ ...
│ ├ pooled_rep                    #   for pooled replicate
│ ├ pseudo_reps                   #   for self pseudo replicates
│ │ ├ rep1                        #    for replicate 1
│ │ │ ├ pr1                       #     for self pseudo replicate 1 of replicate 1
│ │ │ ├ pr2                       #     for self pseudo replicate 2 of replicate 1
│ │ ├ rep2                        #    for repilicate 2
│ │ ...                           
│ └ pooled_pseudo_reps            #   for pooled pseudo replicates
│   ├ ppr1                        #    for pooled pseudo replicate 1 (rep1-pr1 + rep2-pr1 + ...)
│   └ ppr2                        #    for pooled pseudo replicate 2 (rep1-pr2 + rep2-pr2 + ...)
│
├ peak                             #  peaks called
│ └ macs2                          #   peaks generated by MACS2
│   ├ rep1                         #    for replicate 1
│   │ ├ *.narrowPeak.gz            #     narrowPeak (p-val threshold = 0.01)
│   │ ├ *.filt.narrowPeak.gz (E)   #     blacklist filtered narrowPeak 
│   │ ├ *.narrowPeak.bb (E)        #     narrowPeak bigBed
│   │ ├ *.narrowPeak.hammock.gz    #     narrowPeak track for WashU browser
│   │ ├ *.pval0.1.narrowPeak.gz    #     narrowPeak (p-val threshold = 0.1)
│   │ └ *.pval0.1.*K.narrowPeak.gz #     narrowPeak (p-val threshold = 0.1) with top *K peaks
│   ├ rep2                         #    for replicate 2
│   ...
│   ├ pseudo_reps                          #   for self pseudo replicates
│   ├ pooled_pseudo_reps                   #   for pooled pseudo replicates
│   ├ overlap                              #   naive-overlapped peaks
│   │ ├ *.naive_overlap.narrowPeak.gz      #     naive-overlapped peak
│   │ └ *.naive_overlap.filt.narrowPeak.gz #     naive-overlapped peak after blacklist filtering
│   └ idr                           #   IDR thresholded peaks
│     ├ true_reps                   #    for replicate 1
│     │ ├ *.narrowPeak.gz           #     IDR thresholded narrowPeak
│     │ ├ *.filt.narrowPeak.gz (E)  #     IDR thresholded narrowPeak (blacklist filtered)
│     │ └ *.12-col.bed.gz           #     IDR thresholded narrowPeak track for WashU browser
│     ├ pseudo_reps                 #    for self pseudo replicates
│     │ ├ rep1                      #    for replicate 1
│     │ ...
│     ├ optimal_set                 #    optimal IDR thresholded peaks
│     │ └ *.filt.narrowPeak.gz (E)  #     IDR thresholded narrowPeak (blacklist filtered)
│     ├ conservative_set            #    optimal IDR thresholded peaks
│     │ └ *.filt.narrowPeak.gz (E)  #     IDR thresholded narrowPeak (blacklist filtered)
│     ├ pseudo_reps                 #    for self pseudo replicates
│     └ pooled_pseudo_reps          #    for pooled pseudo replicate
│
│   
│ 
├ qc                              #  QC logs
│ ├ *IDR_final.qc                 #   Final IDR QC
│ ├ rep1                          #   for true replicate 1
│ │ ├ *.align.log                 #    Bowtie2 mapping stat log
│ │ ├ *.dup.qc                    #    Picard (or sambamba) MarkDuplicate QC log
│ │ ├ *.pbc.qc                    #    PBC QC
│ │ ├ *.nodup.flagstat.qc         #    Flagstat QC for filtered bam
│ │ ├ *M.cc.qc                    #    Cross-correlation analysis score for tagAlign
│ │ ├ *M.cc.plot.pdf/png          #    Cross-correlation analysis plot for tagAlign
│ │ └ *_qc.html/txt               #    ATAQC report
│ ...
│
├ signal                          #  signal tracks
│ ├ macs2                         #   signal tracks generated by MACS2
│ │ ├ rep1                        #    for true replicate 1 
│ │ │ ├ *.pval.signal.bigwig (E)  #     signal track for p-val
│ │ │ └ *.fc.signal.bigwig   (E)  #     signal track for fold change
│ ...
│ └ pooled_rep                    #   for pooled replicate
│ 
├ report                          # files for HTML report
└ meta                            # text files containing md5sum of output files and other metadata
```

Let's examine how well the reads aligned to the reference saccer3 genome: 

In [ ]:
cat $outdir1/

Now, let's examine how many IDR peaks were called for each sample:

In [ ]:
cat $outdir1/

## Part 4: Creating a merged peak set across all samples for downstream analysis 

Finally, we merge the peaks across all conditions to create a master list of peaks for analysis. To do this, we concatenate the IDR peaks from all experiments, sort them, and merge them. 

We take the output of the processing pipeline from the $AGGREGATE_ANALYSIS directory. This is the same analysis you performed above, but gathered in one location for all experiments conducted. 

In [30]:
cd $AGGREGATE_ANALYSIS_DIR
#Use the "find" command to identify all IDR narrowPeak output files and writ ethem to a file. 

#Now, iterate through the list of narrowPeak files and concatenate them into a single master peak list. 




In [ ]:
#concatenate all .narrowPeak files together 
cat *narrowPeak > all.peaks.bed 

#sort the concatenated file 
bedtools sort -i all.peaks.bed > all.peaks.sorted.bed 

#merge the sorted, concatenated fileto join overlapping peaks 

cat all.peaks.sorted.bed | awk -F '\t' 'BEGIN {{ OFS="\t" }} {{ $2=$2+$10-100; $3=$2+$10+100; if ($2<0) {{$2 = 0}} print $0 }}' | bedtools sort | bedtools merge > all_merged.peaks.bed
gzip -f all_merged.peaks.bed 
